In [56]:
import spacy
import nltk.data
import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field, BucketIterator,LabelField
from torchtext import data
from collections import Counter
import spacy
import numpy as np
import re
import random
import math
import time
import pandas as pd
from sklearn.model_selection import train_test_split

In [57]:
authors=['Hemingway','Nietzsche','Wilde']
authors_id=[0,1,2]
PATH = "/home/julia/PycharmProjects/seq2seq/books/"
books=[]
nlp = spacy.load('en')

TRAIN_SIZE = 0.8
TEST_SIZE = 0.2


In [4]:
def file_open(path):
    #Remove special characters
    with open(path, encoding='utf-8') as f:
        text = f.read()
        #text = re.sub(r'([^a-záéíóúÁÉÍÓÚA-Z0-9,._── :?!¡¿])', '', text)
        return text

In [5]:
def convert_sentences(text):
    file_doc = nlp(text)
    #if len(sentences)>2:
    sentences = list(file_doc.sents)
    return sentences

In [6]:
### Abrir Documentos

sents_by_author=[]
for author in authors:
    name_file = PATH + author
    print(name_file)
    books_author = []
    sents = []
    for file in glob.glob(name_file + "/*.txt"):
        books_author.append(file)
        txt = file_open(file)
        sents = sents+convert_sentences(txt)
    sents_by_author.append(sents)
    #print(books_author)
    books.append(books_author)

/home/julia/PycharmProjects/seq2seq/books/Hemingway
/home/julia/PycharmProjects/seq2seq/books/Nietzsche
/home/julia/PycharmProjects/seq2seq/books/Wilde


In [7]:
print(len(sents_by_author[0])) #número de oraciones para Hemingway
print(len(sents_by_author[1]))#numero de oraciones para Nietzche
print(len(sents_by_author[2])) #número de oraciones de Wilde

18813
20235
24288


In [13]:
row = []
for i in range(len(authors)):
    for sent in sents_by_author[i]:
        if (len(sent)>=3):
            row.append([sent,authors[i]])
         

In [14]:
print(len(row))

59407


In [15]:
df= pd.DataFrame(row,columns=["input", 'author'])

In [55]:
df.head()

NameError: name 'df' is not defined

In [17]:
df.tail()

,input,author
59402,"(At, last, !, \n\n)",Wilde
59403,"(Lady, Bracknell, ., )",Wilde
59404,"(My, nephew, ,, you, seem, to, be, displaying,...",Wilde
59405,"(Jack, ., )",Wilde
59406,"(On, the, contrary, ,, Aunt, Augusta, ,, I, 'v...",Wilde


In [87]:
train_df, test_df = train_test_split(df,
                                         test_size=TEST_SIZE,
                                         train_size=TRAIN_SIZE,
                                         random_state=69,
                                         shuffle=True)

In [98]:
train_df.to_csv("/home/julia/PycharmProjects/seq2seq/data/trainset_classification.csv", index=False)
test_df.to_csv("/home/julia/PycharmProjects/seq2seq/data/testset_classification.csv", index=False)

In [93]:
print(train_df.shape) 
print(test_df.shape)

(50668, 2)
(12668, 2)


In [58]:
def tokenize_input(text):
    return [tok.text for tok in nlp.tokenizer(text)]

In [59]:
SRC = Field(tokenize = tokenize_input,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [60]:
#LABEL = data.LabelField(dtype = torch.float)
 LABEL = data.Field(sequential=False)

In [119]:
PATH = '/home/julia/PycharmProjects/seq2seq/data/'
train = pd.read_csv(PATH+'trainset_classification_biclass.csv')
validation = pd.read_csv(PATH+'testset_classification_biclass.csv')

In [98]:
train_set, test_set = data.TabularDataset.splits(path='/home/julia/PycharmProjects/seq2seq/data/',
                                                train = 'trainset_classification_biclass.csv',
                                                validation = 'testset_classification_biclass.csv',
                                                format='csv',
                                                fields=[('input',SRC),('author',LABEL)],
                                                skip_header=True)

In [99]:
len(train_set)

34671

In [100]:
vars(train_set.examples[7])

{'input': ['"', 'his', 'pants', 'looked', 'mighty', 'like', 'billy', '.', '"'],
 'author': '0'}

# Modelo

In [101]:

class AuthorClassification(nn.Module):
    def __init__(self, vocab_size, embed_dim,embed_matrix,hidden_dim, n_layers,n_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data.copy_(embed_matrix)
        #self.embedding.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embed_dim,hidden_dim,num_layers=n_layers)
        self.linear = nn.Linear(hidden_dim,n_class)
        self.dropout=nn.Dropout(0.2)
        #self.act = nn.Sigmoid()
        #self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        x = self.dropout(embedded)
        lstm_out,(h_t,c_t)=self.rnn(x)
        return self.linear(h_t[-1])

In [102]:
def initialize_Model(vocab_size, embed_dim,embed_matrix,hidden_dim, n_layers,n_class):
    model = AuthorClassification(vocab_size,embed_dim,embed_matrix,hidden_dim,n_layers,n_class)
    return model

## Entrenar modelo
### Parámetros

In [103]:
def get_number_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Función de entrenamiento 

In [104]:
def train(model, iterator, optimizer, criterion,clip_value,training = True):
    if training:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0
    epoch_acc = 0
    loss_sum = 0
    #acc_sum = 0
 
    
    for batch in iterator:
        
        
        predictions = model(batch.text).squeeze(1)        
        loss = criterion(predictions, batch.label)
        
        #acc = binary_accuracy(predictions, batch.label)
        
        
        if training:
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),clip_value)
            optimizer.step()
            
        
        loss_sum += loss.item()
        #acc_sum+=acc
        
        
        
    epoch_loss = loss_sum / len(iterator)   
    #ollasepoch_loss += loss.item()
    #epoch_acc += acc.item()
        
    return epoch_loss / len(iterator)


In [105]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Inicio de Entrenamiento 

In [106]:
device =  'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


###  Crear vocabulario


In [107]:
SRC.build_vocab(train_set,
                         #dev_set,
                         max_size = None,
                         min_freq = 1,
                         vectors = "glove.6B.300d")

In [108]:
LABEL.build_vocab(train_set,)

In [109]:
print(f"\tEncoder embedding matrix: {SRC.vocab.vectors.shape}")

	Encoder embedding matrix: torch.Size([27594, 300])


### Definir Hiperparámetros

In [110]:
INPUT_DIM = len(SRC.vocab)
EMB_DIM = SRC.vocab.vectors.shape[1]
HID_DIM = 512
N_CLASSES=3
N_LAYERS = 2
ENC_DROPOUT = 0.5
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EMB_MATIX = SRC.vocab.vectors
PAD_IDX = SRC.vocab.stoi[SRC.pad_token] # padding

In [111]:
train_iterator, dev_iterator = BucketIterator.splits((train_set, test_set),
                                        batch_size = BATCH_SIZE,
                                        sort_within_batch = True,
                                        #sort_key = lambda x: len(x.src), #attribute the text should be sorted
                                        device = device,
                                        shuffle = True)

In [112]:
vocab_size = INPUT_DIM
embed_dim = EMB_DIM
embed_matrix = EMB_MATIX
hidden_dim = HID_DIM
n_layers = N_LAYERS
n_class = N_CLASSES

In [113]:
model = initialize_Model(vocab_size, embed_dim,embed_matrix,hidden_dim, n_layers,n_class)

In [114]:
model = model.to('cpu')

In [115]:

print(f'El modelo tiene {get_number_parameters(model):,} de parámetros entrenables')

El modelo tiene 12,048,059 de parámetros entrenables


In [116]:
PAD_IDX = SRC.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [117]:
optimizer = optim.Adam(params=model.parameters(),lr = LEARNING_RATE)

# Entrenamiento de modelo

In [118]:
N_EPOCHS = 10
CLIP = 1
p = 1.0

best_valid_loss = float('inf')

train_loss_epoch = []
test_loss_epoch = []

for epoch in range(N_EPOCHS):
    #p = max(min_prob, (p - linear_decay_num)) 
    start_time = time.time()
    #train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    train_loss = train(model=model,
                       iterator=train_iterator,
                       #teacher_forcing_prob = p,
                       optimizer=optimizer,
                       criterion=criterion,
                       clip_value=CLIP,
                       training = True)
    #test_loss = evaluate(model, test_iterator, criterion)
    """with torch.no_grad():
        test_loss = train(model=model,
                         iterator=test_iterator,
                         optimizer=None,
                         criterion=criterion,
                         clip=CLIP)
    """
    train_loss_epoch.append(train_loss)
    #test_loss_epoch.append(test_loss)

    #valid_loss = evaluate(model, test_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print('-' * 89)
    print(f'Epoch: {epoch+1:03} | Time: {epoch_mins}m {epoch_secs}s | TFprob: {p:.4f} | Train loss: {train_loss:.4f} | ')
print('-' * 89)

    #if test_loss < best_valid_loss:
    #    best_valid_loss = test_loss
    #    torch.save(model.state_dict(), 'tut001-model.pt')

TypeError: '<' not supported between instances of 'Example' and 'Example'

In [50]:
def train(model, iterator ,optimizer, criterion, clip):
    model.train()

    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        optimizer.zero_grad()
        output = model(src)
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        # trg = [(trg len - 1) * batch size]
        # output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [51]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs